# ステップ06 手作業によるバックプロパゲーション

Variableクラスはgradにnothingを許すようにすることで未定義の状況を可能にする

In [4]:
mutable struct Variable
    data::Array{Float64}
    grad::Union{Array{Float64},Nothing}
    Variable(data) = new(data, nothing)
end

abstract type AbstractFunction end

Pythonでは親クラスに実装していたコードは、相当するabstract typeをthisに取る関数として独立。やり方は違うがコードの重複がないことは同じ。

In [5]:
function call!(this::AbstractFunction, input::Variable)
    x = input.data
    y = forward(this, x)
    output = Variable(y)
    this.input = input
    return output
end

call! (generic function with 1 method)

ここではinputにnothingを許す。またforwardやbackwardはそれぞれの型に応じてmultiple dispatchがかかるようにしておく。OOPっぽい考え方から離れる。

In [6]:
mutable struct Square <: AbstractFunction
    input::Union{Variable,Nothing}
    Square() = new(nothing)
end

function forward(this::Square, x::Array{Float64})
    y = x .^ 2
    return y
end

function backward(this::Square, gy::Array{Float64})
    x = this.input.data
    gx = 2 .* x .* gy
    return gx
end

backward (generic function with 1 method)

In [7]:
mutable struct Exp <: AbstractFunction
    input::Union{Variable,Nothing}
    Exp() = new(nothing)
end

function forward(this::Exp, x::Array{Float64})
    y = exp.(x)
    return y
end

function backward(this::Exp, gy::Array{Float64})
    x = this.input.data
    gx = exp.(x) .* gy
    return gx
end

backward (generic function with 2 methods)

In [8]:
A = Square();
B = Exp();
C = Square();

In [10]:
x = Variable([0.5])

Variable([0.5], nothing)

In [11]:
a = call!(A, x)

Variable([0.25], nothing)

In [12]:
b = call!(B, a)

Variable([1.2840254166877414], nothing)

In [13]:
y = call!(C, b)

Variable([1.648721270700128], nothing)

In [15]:
y.grad = [1.]

1-element Array{Float64,1}:
 1.0

In [16]:
b.grad = backward(C, y.grad)

1-element Array{Float64,1}:
 2.568050833375483

In [17]:
a.grad = backward(B, b.grad)

1-element Array{Float64,1}:
 3.297442541400256

In [18]:
x.grad = backward(A, a.grad)

1-element Array{Float64,1}:
 3.297442541400256